<div style="font-size:40px; font-weight:bold; margin:20px; margin-bottom:100px; text-align: justify;text-shadow: 1px 1px 1px #919191,
        1px 2px 1px #919191,
        1px 3px 1px #919191,
        1px 4px 1px #919191,
        1px 5px 1px #919191,
        1px 6px 1px #919191,
        1px 7px 1px #919191,
        1px 8px 1px #919191,
        1px 9px 1px #919191,
        1px 10px 1px #919191,
    1px 18px 6px rgba(16,16,16,0.4),
    1px 22px 10px rgba(16,16,16,0.2),
    1px 25px 35px rgba(16,16,16,0.2),
    1px 30px 60px rgba(16,16,16,0.4)">Long Short Term Memory Recurrent Networks</div>

<div style="font-style: italic; font-weight: bold; font-size:35px; text-align:center; font-family: Garamond">by Rubén Cañadas Rodríguez</div>

<div style="font-size: 30px; margin: 20px; margin-bottom: 40px; margin-left: 0px; line-height: 40pt">

<div style="font-size: 30px; font-family: Garamond; font-weight: bold; margin: 30px; margin-left: 0px; margin-bottom: 10px; ">Contents</div>
<ol>
<li>Introduction</li>
<li>Recurrent Neural Networks (RNNs)</li>
<li>Long Short Term Memory (LSTM)</li> 
<li>Text generation</li> 
<li>Coding</li> 
</ol>
</div>
<div style="font-size: 30px; font-weight: bold; margin-bottom: 20px; margin-top: 30px"> Introduction </div>
<div style="font-size: 30px; font-weight: bold; margin-bottom: 20px; margin-top: 30px"> Recurrent Neural Networks (RNNs) </div>
<div style="font-size: 30px; font-weight: bold; margin-bottom: 20px; margin-top: 30px"> Long Short Term Memory (LSTM) </div>
<div style="font-size: 30px; font-weight: bold; margin-bottom: 20px; margin-top: 30px"> Text generation </div>
<div style="font-size: 30px; font-weight: bold; margin-bottom: 20px; margin-top: 30px"> Coding </div>

<div style="font-size: 40px; font-weight: bold; text-align: left">Import packages and modules</div>
<br>
<br>


In [2]:
import pandas as pd
import re #regular expressions for treating text
import numpy as np
from nltk import word_tokenize #Natural Language Processing package

In [8]:
#Loading csv
CSV_NAME = "QUOTE.csv"
df = pd.read_csv(CSV_NAME)
quotes = list(df.quote + "\n") #adding a break at the end of each quote!


In [36]:
removed_char = ['#', '$', '%', '(', ')', '=', ';' ,':',  '*', '+', '£' , '—','’'] #Characters that we dont want to
#include in our model and we just remove them!
quotes_cleaned = []

for quote in quotes: 
    # remove unused character
    for s_char in removed_char:
        quote = quote.replace(s_char, ' ')
    
    # remove white space
    pattern = re.compile(r'\s{2,}') #regular expression for replacing more than two white spaces
    quote = re.sub(pattern, ' ', quote)

    quotes_cleaned.append(quote)

text = ' '.join(quotes_cleaned)
chars = sorted(list(set(text))) #Chars are a list of characters found in the quotes (letters + symbols)
char_indices = dict((c, i) for i, c in enumerate(chars)) #To each character we assign a number
indices_char = dict((i, c) for i, c in enumerate(chars)) #The contrary, to each number we asssign a character
print("indices char ", char_indices)


indices char  {'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'X': 44, 'Y': 45, 'Z': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72}


In [23]:

maxlen = 15 # Lenght of the new sentences
step = 6
sentences = []
next_chars = []

for quote in quotes_cleaned:
    for i in range(0, len(quote) - maxlen, step):
        sentences.append(quote[i: i + maxlen])
        next_chars.append(quote[i + maxlen])
    sentences.append(quote[-maxlen:])
    next_chars.append(quote[-1])
sentences = sentences[:100]
sentences[:20]

['If you live to ',
 ' live to be a h',
 'to be a hundred',
 'a hundred, I wa',
 'red, I want to ',
 ' want to live t',
 'to live to be a',
 'e to be a hundr',
 'e a hundred min',
 'ndred minus one',
 'minus one day s',
 'one day so I ne',
 'y so I never ha',
 ' never have to ',
 ' have to live w',
 'to live without',
 'e without you.\n',
 "Promise me you'",
 "e me you'll alw",
 "ou'll always re"]

In [38]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
   

Vectorization...


<table style="width:100%; margin: 20px; margin-left:-300px">
  <tr>
    <th>Predictors (X train)</th>
    <th>Labels (Y train)</th>
  </tr>
  <tr>
    <td>they</td>
    <td>are</td>
  </tr>
  <tr>
    <td>they are</td>
    <td>learning</td>
  </tr>
  <tr>
    <td>they are learning</td>
    <td>artificial</td>
  </tr>
      <tr>
    <td>they are learning artificial</td>
    <td>inteligence</td>
  </tr>
</table>